In [24]:
# coding:utf-8
from __future__ import print_function
from random import shuffle
from past.builtins import xrange
import pickle
import numpy as np

# 文件读取
def unpickle(file):
	with open(file,'rb') as fo:
		dict = pickle.load(fo)
	return dict

def load_file(file):
	dictTrain = unpickle(file + "data_batch_1")
	dataTrain = dictTrain['data']
	labelTrain = dictTrain['labels']

	for i in range(2,6):
		dictTrain = unpickle(file + "data_batch_" + str(i))
		dataTrain = np.vstack([dataTrain,dictTrain['data']])
		labelTrain = np.hstack([labelTrain,dictTrain['labels']])

	dictTest = unpickle(file + "test_batch")
	dataTest = dictTest['data']
	labelTest = dictTest['labels']
	labelTest = np.array(labelTest)

	return dataTrain, labelTrain, dataTest, labelTest


#softmax loss 函数
def softmax_loss_naive(W, X, y, reg):
	'''
		W:权重矩阵
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
	'''
	#初始化数据
	loss = 0.0
	dW = np.zeros_like(W)
	num_train = X.shape[0]	#样本数
	num_class = W.shape[1]	#样本类别数

	for i in xrange(num_train):
		score = X[i].dot(W)
		score -= np.max(score)	#提高样本稳定性

		correct_score = score[y[i]]
		exp_sum = np.sum(np.exp(score))
		loss += np.log(exp_sum) - correct_score

		for j in xrange(num_class):
			if (j == y[i]):
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i] - X[i]
			else:
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i]


	loss /= num_train
	loss += 0.5 * reg * np.sum(W*W)

	dW /= num_train
	dW += reg * W

	return loss, dW

#线性分类器
class LinearClassifier(object):
	def __init__(self):
		self.W = None

	def train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True):
		'''
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		step_size:学习步进速度
		reg:正则化强度
		num_iters:迭代次数
		batch_size:每次迭代图片样本数
		verbose:是否打印信息

		return:
			loss_history:每次训练loss值
		'''
		num_train, dim = X.shape
		num_classes = np.max(y) + 1
	
		if self.W is None:
			self.W = 0.001 * np.random.randn(dim, num_classes)

		loss_history = []

		for it in xrange(num_iters):
			#从样本中不重复随机采batch_size个样本
			sample_index = np.random.choice(num_train, batch_size, replace=False)

			X_batch = X[sample_index, :]
			y_batch = y[sample_index]

			loss, grad = self.loss(X_batch, y_batch, reg)
			loss_history.append(loss)

			self.W += -step_size * grad

			if (verbose and it %10 == 0):
				print('iteration %d / %d, samples: %d, loss: %f' % (it, num_iters, batch_size, loss))

		return loss_history

	def predict(self, X):
		'''
		X:图片训练集(矩阵)

		return:
			y_pred:标签预测值
		'''
		y_pred = np.zeros(X.shape[1])

		score = X.dot(self.W)
		y_pred = np.argmax(score, axis = 1)

		return y_pred


	def loss(self, X_batch, y_batch, reg):
		'''
		X_batch:图片训练集(矩阵)
		y_batch:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
		'''
		return softmax_loss_naive(self.W, X_batch, y_batch, reg)

In [25]:
from skimage.feature import local_binary_pattern
from PIL import Image
#开始训练
file_path = './'

dataTrFirst, labelTrain, dataTsFirst, labelTest = load_file(file_path)

dataTr = np.zeros((dataTrFirst.shape[0],32*32))
dataTs = np.zeros((dataTsFirst.shape[0],32*32))


for i in range(dataTrFirst.shape[0] -45000):
    img = dataTrFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTr[i] = res.reshape((1,32*32))
print("训练集加载完成")

for i in range(dataTsFirst.shape[0] -1):
    img = dataTsFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTs[i] = res.reshape((1,32*32))
print("测试集加载完成")

dataTrain = dataTrain - np.mean(dataTrain, axis=0)


训练集加载完成
测试集加载完成


In [ ]:
LC = LinearClassifier()

print('start training ...')
#train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True)
#在dataTrain中不重复随机抽取batch_size个样本，迭代训练num_iters次
loss_all = LC.train(dataTrain, labelTrain, num_iters = 8000, batch_size = 200)


start training ...
iteration 0 / 8000, samples: 200, loss: 4.971027
iteration 10 / 8000, samples: 200, loss: 384.378958
iteration 20 / 8000, samples: 200, loss: 293.225315
iteration 30 / 8000, samples: 200, loss: 215.186162
iteration 40 / 8000, samples: 200, loss: 309.724507
iteration 50 / 8000, samples: 200, loss: 431.009159
iteration 60 / 8000, samples: 200, loss: 242.300164
iteration 70 / 8000, samples: 200, loss: 265.746932
iteration 80 / 8000, samples: 200, loss: 452.359751
iteration 90 / 8000, samples: 200, loss: 333.418578
iteration 100 / 8000, samples: 200, loss: 163.394712
iteration 110 / 8000, samples: 200, loss: 230.101835
iteration 120 / 8000, samples: 200, loss: 421.908613
iteration 130 / 8000, samples: 200, loss: 304.555362
iteration 140 / 8000, samples: 200, loss: 305.773754
iteration 150 / 8000, samples: 200, loss: 213.665798
iteration 160 / 8000, samples: 200, loss: 255.989628
iteration 170 / 8000, samples: 200, loss: 203.012875
iteration 180 / 8000, samples: 200, loss

iteration 1540 / 8000, samples: 200, loss: 218.681627
iteration 1550 / 8000, samples: 200, loss: 239.035256
iteration 1560 / 8000, samples: 200, loss: 249.448818
iteration 1570 / 8000, samples: 200, loss: 298.953567
iteration 1580 / 8000, samples: 200, loss: 174.608600
iteration 1590 / 8000, samples: 200, loss: 375.521898
iteration 1600 / 8000, samples: 200, loss: 249.784439
iteration 1610 / 8000, samples: 200, loss: 278.854722
iteration 1620 / 8000, samples: 200, loss: 299.617413
iteration 1630 / 8000, samples: 200, loss: 227.050258
iteration 1640 / 8000, samples: 200, loss: 350.381114
iteration 1650 / 8000, samples: 200, loss: 139.918941
iteration 1660 / 8000, samples: 200, loss: 457.544565
iteration 1670 / 8000, samples: 200, loss: 174.932525
iteration 1680 / 8000, samples: 200, loss: 297.269129
iteration 1690 / 8000, samples: 200, loss: 264.610141
iteration 1700 / 8000, samples: 200, loss: 175.602068
iteration 1710 / 8000, samples: 200, loss: 296.329645
iteration 1720 / 8000, sampl

iteration 3060 / 8000, samples: 200, loss: 270.662928
iteration 3070 / 8000, samples: 200, loss: 186.969014
iteration 3080 / 8000, samples: 200, loss: 203.377238
iteration 3090 / 8000, samples: 200, loss: 340.780691
iteration 3100 / 8000, samples: 200, loss: 195.289555
iteration 3110 / 8000, samples: 200, loss: 303.140388
iteration 3120 / 8000, samples: 200, loss: 318.293030
iteration 3130 / 8000, samples: 200, loss: 204.679222
iteration 3140 / 8000, samples: 200, loss: 301.844122
iteration 3150 / 8000, samples: 200, loss: 166.949842
iteration 3160 / 8000, samples: 200, loss: 278.239741
iteration 3170 / 8000, samples: 200, loss: 311.591996
iteration 3180 / 8000, samples: 200, loss: 264.091710
iteration 3190 / 8000, samples: 200, loss: 288.262821
iteration 3200 / 8000, samples: 200, loss: 226.270738
iteration 3210 / 8000, samples: 200, loss: 271.707280
iteration 3220 / 8000, samples: 200, loss: 235.208952
iteration 3230 / 8000, samples: 200, loss: 304.182410
iteration 3240 / 8000, sampl

iteration 4580 / 8000, samples: 200, loss: 234.901068
iteration 4590 / 8000, samples: 200, loss: 274.888867
iteration 4600 / 8000, samples: 200, loss: 263.968712
iteration 4610 / 8000, samples: 200, loss: 217.467367
iteration 4620 / 8000, samples: 200, loss: 195.028329
iteration 4630 / 8000, samples: 200, loss: 251.966592
iteration 4640 / 8000, samples: 200, loss: 175.003822
iteration 4650 / 8000, samples: 200, loss: 375.231832
iteration 4660 / 8000, samples: 200, loss: 253.186012
iteration 4670 / 8000, samples: 200, loss: 287.192606
iteration 4680 / 8000, samples: 200, loss: 452.944437
iteration 4690 / 8000, samples: 200, loss: 237.906459
iteration 4700 / 8000, samples: 200, loss: 250.504308
iteration 4710 / 8000, samples: 200, loss: 228.055683
iteration 4720 / 8000, samples: 200, loss: 221.777141
iteration 4730 / 8000, samples: 200, loss: 349.652011
iteration 4740 / 8000, samples: 200, loss: 277.871908
iteration 4750 / 8000, samples: 200, loss: 323.730156
iteration 4760 / 8000, sampl

iteration 6100 / 8000, samples: 200, loss: 212.525544
iteration 6110 / 8000, samples: 200, loss: 218.346935
iteration 6120 / 8000, samples: 200, loss: 267.664083
iteration 6130 / 8000, samples: 200, loss: 171.299013
iteration 6140 / 8000, samples: 200, loss: 294.860152
iteration 6150 / 8000, samples: 200, loss: 210.471417
iteration 6160 / 8000, samples: 200, loss: 249.203731
iteration 6170 / 8000, samples: 200, loss: 243.805420
iteration 6180 / 8000, samples: 200, loss: 335.455424
iteration 6190 / 8000, samples: 200, loss: 276.318166
iteration 6200 / 8000, samples: 200, loss: 294.882872
iteration 6210 / 8000, samples: 200, loss: 178.112571
iteration 6220 / 8000, samples: 200, loss: 164.200721
iteration 6230 / 8000, samples: 200, loss: 400.979804
iteration 6240 / 8000, samples: 200, loss: 265.005544
iteration 6250 / 8000, samples: 200, loss: 227.078978
iteration 6260 / 8000, samples: 200, loss: 250.541992
iteration 6270 / 8000, samples: 200, loss: 276.402338
iteration 6280 / 8000, sampl

In [ ]:
dataTest = dataTest - np.mean(dataTest, axis=0)
print('last loss is %f' %(loss_all[-1]))
#开始预测
print('start predicting ...')
y_pred = LC.predict(dataTest)

hit = 0
for i in xrange(y_pred.size):
	if (y_pred[i] == labelTest[i]):
		hit += 1

print('the accuracy rate is %f ' % (hit/100))